In [10]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import datetime
import time
import sys

sys.path.append("/Users/derekdewald/Documents/Python/Github_Repo/d_py_functions")

from TextFunctions import extract_doc_sections_all
from MLPipeline import SKLearnModelList

def GenerateSKModelDoc():

    sklearn_model_df = SKLearnModelList()
       
    param_df = pd.DataFrame()
    desc_df = pd.DataFrame()

    for index,row in sklearn_model_df.iterrows():
        temp_para,temp_desc = extract_doc_sections_all(row['Estimator Class'],model_name=row['Model Name'])
        param_df = pd.concat([param_df,temp_para])
        desc_df = pd.concat([desc_df,temp_desc])

    sklearn_model_df = sklearn_model_df.merge(desc_df,on='Model Name',how='left') 

    sklearn_model_df.to_csv('SKLearnModels.csv',index=False)
    param_df.to_csv('SKlearnParameterList.csv',index=False)
    
    return sklearn_model_df,param_df

sklearn_model_df,param_df = GenerateSKModelDoc()


NameError: name 'b' is not defined

In [ ]:

from UtilityFunctions import InspectFunction

InspectFunction(extract_doc_sections_all)

In [16]:
desc_df

,Model Name,Sklearn Desc
0,Bayesian ARD regression.,Bayesian ARD regression. Fit the weights of a...
0,An AdaBoost classifier.,An AdaBoost classifier. An AdaBoost [1]_ clas...
0,An AdaBoost regressor.,An AdaBoost regressor. An AdaBoost [1] regres...
0,Approximate feature map for additive chi2 kernel.,Approximate feature map for additive chi2 kern...
0,Perform Affinity Propagation Clustering of data.,Perform Affinity Propagation Clustering of dat...
...,...,...
0,Classifier that post-tunes the decision thresh...,Classifier that post-tunes the decision thresh...
0,Generalized Linear Model with a Tweedie distri...,Generalized Linear Model with a Tweedie distri...
0,Feature selector that removes all low-variance...,Feature selector that removes all low-variance...
0,Soft Voting/Majority Rule classifier for unfit...,Soft Voting/Majority Rule classifier for unfit...


In [4]:
sklearn_model_df

,Model Name,Estimator Class,Dataset Size,Full Class Path,Estimator Type,Part_1,Part_2,Part_3,Part_4,Part_5
0,ARDRegression,<class 'sklearn.linear_model._bayes.ARDRegress...,"Small, Medium",sklearn.linear_model._bayes.ARDRegression,regressor,sklearn,linear_model,_bayes,ARDRegression,None
1,AdaBoostClassifier,<class 'sklearn.ensemble._weight_boosting.AdaB...,"Small, Medium",sklearn.ensemble._weight_boosting.AdaBoostClas...,classifier,sklearn,ensemble,_weight_boosting,AdaBoostClassifier,None
2,AdaBoostRegressor,<class 'sklearn.ensemble._weight_boosting.AdaB...,"Small, Medium",sklearn.ensemble._weight_boosting.AdaBoostRegr...,regressor,sklearn,ensemble,_weight_boosting,AdaBoostRegressor,None
3,AdditiveChi2Sampler,<class 'sklearn.kernel_approximation.AdditiveC...,Large,sklearn.kernel_approximation.AdditiveChi2Sampler,transformer,sklearn,kernel_approximation,AdditiveChi2Sampler,None,None
4,AffinityPropagation,<class 'sklearn.cluster._affinity_propagation....,Small,sklearn.cluster._affinity_propagation.Affinity...,cluster,sklearn,cluster,_affinity_propagation,AffinityPropagation,None
...,...,...,...,...,...,...,...,...,...,...
212,TunedThresholdClassifierCV,<class 'sklearn.model_selection._classificatio...,Unknown,sklearn.model_selection._classification_thresh...,classifier,sklearn,model_selection,_classification_threshold,TunedThresholdClassifierCV,None
213,TweedieRegressor,<class 'sklearn.linear_model._glm.glm.TweedieR...,Medium,sklearn.linear_model._glm.glm.TweedieRegressor,regressor,sklearn,linear_model,_glm,glm,TweedieRegressor
214,VarianceThreshold,<class 'sklearn.feature_selection._variance_th...,Large,sklearn.feature_selection._variance_threshold....,transformer,sklearn,feature_selection,_variance_threshold,VarianceThreshold,None
215,VotingClassifier,<class 'sklearn.ensemble._voting.VotingClassif...,"Medium, Large",sklearn.ensemble._voting.VotingClassifier,"classifier, transformer",sklearn,ensemble,_voting,VotingClassifier,None


In [ ]:

from DataSets import MNIST_SKLEARN
from MLPipeline import MLPipelineSample

df,z = MNIST_SKLEARN(normalize=False,flatten=True,return_value='df')

results_df = MLPipelineSample(df,
                              scaler='normal',
                              ml_model_type='classifier',
                              target_column='Target',
                              test_size=0.2)

In [ ]:
# Need to Add ability to filter ML Models Based on Dataset size.
# Hard Code Small, Medium Large, 
# Add new columns and helpful hitns to Datasheet

In [32]:
results_df = final_df.copy()

In [156]:
a.head()

,Model Name,Estimator Class,Dataset Size,Full Class Path,Estimator Type,Part_1,Part_2,Part_3,Part_4,Part_5
0,ARDRegression,<class 'sklearn.linear_model._bayes.ARDRegress...,"Small, Medium",sklearn.linear_model._bayes.ARDRegression,regressor,sklearn,linear_model,_bayes,ARDRegression,None
1,AdaBoostClassifier,<class 'sklearn.ensemble._weight_boosting.AdaB...,"Small, Medium",sklearn.ensemble._weight_boosting.AdaBoostClas...,classifier,sklearn,ensemble,_weight_boosting,AdaBoostClassifier,None
2,AdaBoostRegressor,<class 'sklearn.ensemble._weight_boosting.AdaB...,"Small, Medium",sklearn.ensemble._weight_boosting.AdaBoostRegr...,regressor,sklearn,ensemble,_weight_boosting,AdaBoostRegressor,None
3,AdditiveChi2Sampler,<class 'sklearn.kernel_approximation.AdditiveC...,Large,sklearn.kernel_approximation.AdditiveChi2Sampler,transformer,sklearn,kernel_approximation,AdditiveChi2Sampler,None,None
4,AffinityPropagation,<class 'sklearn.cluster._affinity_propagation....,Small,sklearn.cluster._affinity_propagation.Affinity...,cluster,sklearn,cluster,_affinity_propagation,AffinityPropagation,None


In [165]:
print(a['Estimator Class'].iloc[0].__doc__)

Bayesian ARD regression.

    Fit the weights of a regression model, using an ARD prior. The weights of
    the regression model are assumed to be in Gaussian distributions.
    Also estimate the parameters lambda (precisions of the distributions of the
    weights) and alpha (precision of the distribution of the noise).
    The estimation is done by an iterative procedures (Evidence Maximization)

    Read more in the :ref:`User Guide <bayesian_regression>`.

    Parameters
    ----------
    max_iter : int, default=300
        Maximum number of iterations.

        .. versionchanged:: 1.3

    tol : float, default=1e-3
        Stop the algorithm if w has converged.

    alpha_1 : float, default=1e-6
        Hyper-parameter : shape parameter for the Gamma distribution prior
        over the alpha parameter.

    alpha_2 : float, default=1e-6
        Hyper-parameter : inverse scale parameter (rate parameter) for the
        Gamma distribution prior over the alpha parameter.

    lambda

In [149]:
from MLPipeline import SKLearnModelList


def test(estimator_class):
    """
    Extracts parameter names and descriptions from a scikit-learn estimator's docstring.
    Parameters:
        estimator_class: A scikit-learn class (not an instance), e.g., sklearn.linear_model.LogisticRegression
    Returns:
        pd.DataFrame with columns ['Parameter', 'Description']
    """
    doc = estimator_class.__doc__
    print(doc)
 

In [139]:
a

In [141]:
b

'An AdaBoost classifier.  An AdaBoost [1]_ classifier is a meta-estimator that begins by fitting a classifier on the original dataset and then fits additional copies of the classifier on the same dataset but where the weights of incorrectly classified instances are adjusted such that subsequent classifiers focus more on difficult cases.  This class implements the algorithm based on [2]_.  Read more in the :ref:`User Guide <adaboost>`.'

In [102]:
import pandas as pd
import re

def ExtractSklearnParamsDoc(estimator_class,
                            breakpoints=['Parameters','Attributes','See Also','Notes','References','Examples'],
                            return_value='python_function'):
    """
    Extracts structured parameter and attribute docs from a scikit-learn estimator's docstring.

    Parameters:
        estimator_class: A scikit-learn class (not an instance)
        breakpoints: Section headers to identify in docstring
        return_value: Option to format the return (currently only 'python_function' is supported)

    Returns:
        pd.DataFrame with columns: ['Section', 'Name', 'Type', 'Default', 'Description']
    """
    doc = estimator_class.__doc__
    lines = [x.strip() for x in doc.split('\n') if x.strip()]
    
    results = []

    if return_value == 'python_function':
        for count, break_ in enumerate(breakpoints):
            try:
                # Get section bounds
                section_start = lines.index(break_)
                section_end = next((lines.index(b) for b in breakpoints[count+1:] if b in lines), len(lines))

                section_lines = lines[section_start + 1:section_end]
                section = break_

                # Parse section content
                current_entry = None
                description_lines = []

                for line in section_lines:
                    if '----' in line or line.startswith('..'):
                        continue

                    # New field line with ':'
                    if ':' in line and not line.startswith(' ') and line.find(':') < 25:
                        if current_entry:
                            results.append(current_entry)

                        # Extract param name/type/default
                        name_part, desc_part = line.split(':', 1)
                        name = name_part.strip()
                        type_match = re.search(r'([^,]+)', desc_part.strip())
                        default_match = re.search(r'default\s*=\s*([^\s,]+)', desc_part)

                        current_entry = {
                            'Section': section,
                            'Name': name,
                            'Type': type_match.group(1).strip() if type_match else '',
                            'Default': default_match.group(1).strip() if default_match else '',
                            'Description': ''
                        }
                        description_lines = []
                    else:
                        # Continuation of description
                        description_lines.append(line.strip())
                        if current_entry:
                            current_entry['Description'] = ' '.join(description_lines)

                # Add last item
                if current_entry:
                    results.append(current_entry)

            except ValueError:
                continue

    return pd.DataFrame(results)
ExtractSklearnParamsDoc(list_[1])

,Section,Name,Type,Default,Description
0,Parameters,estimator,object,None,The base estimator from which the boosted ense...
1,Parameters,the base estimator is,class:`~sklearn.tree.DecisionTreeClassifier`,,initialized with `max_depth=1`. `base_estimato...
2,Parameters,n_estimators,int,50,The maximum number of estimators at which boos...
3,Parameters,learning_rate,float,1.0,Weight applied to each classifier at each boos...
4,Parameters,algorithm,{'SAMME'},'SAMME',Use the SAMME discrete boosting algorithm. `al...
5,Parameters,random_state,int,None,Controls the random seed given at each `estima...
6,Parameters,See,term:`Glossary <random_state>`.,,
7,Attributes,estimator_,estimator,,The base estimator from which the ensemble is ...
8,Attributes,estimators_,list of classifiers,,The collection of fitted sub-estimators.
9,Attributes,classes_,ndarray of shape (n_classes,,The classes labels.


In [93]:
import re
import pandas as pd

def ExtractSklearnParamsDoc(estimator_class,
                            breakpoints=['Parameters','Attributes','See Also','Notes','References','Examples'],
                            return_value='python_function'):
    """
    Extracts parameter names and descriptions from a scikit-learn estimator's docstring.

    Parameters:
        estimator_class: A scikit-learn class (not an instance), e.g., sklearn.linear_model.LogisticRegression

    Returns:
        pd.DataFrame with columns ['Parameter', 'Description']
    """
    doc = estimator_class.__doc__

    split1 = doc.split('\n')

    final = [x.strip() for x in split1 if len(x) != 0]
    
    if return_value=='python_function':

        model = final[0]
        model_desc = final[1]
        
        for count,break_ in enumerate(breakpoints):
        
            # Iterate through to Find Parameters.
            
            try:
                param_start = final.index(break_)
                param_end = final.index(breakpoints[count+1])
                print(final[param_start],final[param_end])
                temp_list = final[final[param_start]+1,final[param_end]+1]
                
                
    
            except:
                pass

    return final

ExtractSklearnParamsDoc(list_[0])

Parameters Attributes
Attributes See Also
See Also Notes
Notes References
References Examples


['Bayesian ARD regression.',
 'Fit the weights of a regression model, using an ARD prior. The weights of',
 'the regression model are assumed to be in Gaussian distributions.',
 'Also estimate the parameters lambda (precisions of the distributions of the',
 'weights) and alpha (precision of the distribution of the noise).',
 'The estimation is done by an iterative procedures (Evidence Maximization)',
 'Read more in the :ref:`User Guide <bayesian_regression>`.',
 'Parameters',
 '----------',
 'max_iter : int, default=300',
 'Maximum number of iterations.',
 '.. versionchanged:: 1.3',
 'tol : float, default=1e-3',
 'Stop the algorithm if w has converged.',
 'alpha_1 : float, default=1e-6',
 'Hyper-parameter : shape parameter for the Gamma distribution prior',
 'over the alpha parameter.',
 'alpha_2 : float, default=1e-6',
 'Hyper-parameter : inverse scale parameter (rate parameter) for the',
 'Gamma distribution prior over the alpha parameter.',
 'lambda_1 : float, default=1e-6',
 'Hyper

In [61]:
f

[('ref',
  'Parameters\n    ----------\n    max_iter : int, default=300\n        Maximum number of iterations.\n\n        .. versionchanged:: 1.3\n\n    tol : float, default=1e-3\n        Stop the algorithm if w has converged.\n\n    alpha_1 : float, default=1e-6\n        Hyper-parameter : shape parameter for the Gamma distribution prior\n        over the alpha parameter.\n\n    alpha_2 : float, default=1e-6\n        Hyper-parameter : inverse scale parameter (rate parameter) for the\n        Gamma distribution prior over the alpha parameter.\n\n    lambda_1 : float, default=1e-6\n        Hyper-parameter : shape parameter for the Gamma distribution prior\n        over the lambda parameter.\n\n    lambda_2 : float, default=1e-6\n        Hyper-parameter : inverse scale parameter (rate parameter) for the\n        Gamma distribution prior over the lambda parameter.\n\n    compute_score : bool, default=False\n        If True, compute the objective function at each step of the model.\n\n    t

In [49]:
def ReadStr(text):
    split_words = text.split(


text

'Bayesian ARD regression.\n\n    Fit the weights of a regression model, using an ARD prior. The weights of\n    the regression model are assumed to be in Gaussian distributions.\n    Also estimate the parameters lambda (precisions of the distributions of the\n    weights) and alpha (precision of the distribution of the noise).\n    The estimation is done by an iterative procedures (Evidence Maximization)\n\n    Read more in the :ref:`User Guide <bayesian_regression>`.\n\n    Parameters\n    ----------\n    max_iter : int, default=300\n        Maximum number of iterations.\n\n        .. versionchanged:: 1.3\n\n    tol : float, default=1e-3\n        Stop the algorithm if w has converged.\n\n    alpha_1 : float, default=1e-6\n        Hyper-parameter : shape parameter for the Gamma distribution prior\n        over the alpha parameter.\n\n    alpha_2 : float, default=1e-6\n        Hyper-parameter : inverse scale parameter (rate parameter) for the\n        Gamma distribution prior over the a

In [46]:
text = model.__doc__

In [43]:
model = list_[0]()
model.get_params()

{'alpha_1': 1e-06,
 'alpha_2': 1e-06,
 'compute_score': False,
 'copy_X': True,
 'fit_intercept': True,
 'lambda_1': 1e-06,
 'lambda_2': 1e-06,
 'max_iter': 300,
 'threshold_lambda': 10000.0,
 'tol': 0.001,
 'verbose': False}

In [34]:
results_df[results_df['Class']=='Macro'].sort_values('AUC',ascending=False)

,Class,TP,FP,FN,TN,Precision,Recall,F1,Accuracy,AUC,Time,Model,Scaler
Total,Macro,197.9,12.1,12.1,1877.9,0.942513,0.941651,0.941923,0.988476,0.997232,1.470992,ExtraTreesClassifier,normal
Total,Macro,197.5,12.5,12.5,1877.5,0.939750,0.939178,0.939368,0.988095,0.997048,8.850870,MLPClassifier,normal
Total,Macro,196.5,13.5,13.5,1876.5,0.935101,0.934708,0.934728,0.987143,0.996516,2.468268,RandomForestClassifier,normal
Total,Macro,192.4,17.6,17.6,1872.4,0.915638,0.915830,0.915646,0.983238,0.994098,187.085119,GradientBoostingClassifier,normal
Total,Macro,188.8,21.2,21.2,1868.8,0.897440,0.897280,0.897253,0.979810,0.990278,18.047525,LogisticRegressionCV,normal
Total,Macro,185.1,24.9,24.9,1865.1,0.880480,0.880222,0.879759,0.976286,0.983628,5.545704,BaggingClassifier,normal
Total,Macro,183.6,26.4,26.4,1863.6,0.873130,0.871803,0.872083,0.974857,0.983250,4.310342,CalibratedClassifierCV,normal
Total,Macro,172.4,37.6,37.6,1852.4,0.822206,0.818228,0.818737,0.964190,0.979861,0.058774,BernoulliNB,normal
Total,Macro,176.1,33.9,33.9,1856.1,0.839375,0.835401,0.835722,0.967714,0.978212,0.734467,LinearDiscriminantAnalysis,normal
Total,Macro,170.1,39.9,39.9,1850.1,0.814382,0.807418,0.808286,0.962000,0.976348,0.015738,MultinomialNB,normal


In [10]:
a.head(15)

,Class,TP,FP,FN,TN,Precision,Recall,F1,Accuracy,AUC,Model,Scaler,Time
0,0,143,25,68,1864,0.851190,0.677725,0.754617,0.955714,0.926316,AdaBoostClassifier,normal,3.633409
1,1,190,18,42,1850,0.913462,0.818966,0.863636,0.971429,0.926316,AdaBoostClassifier,normal,3.633409
2,2,100,82,88,1830,0.549451,0.531915,0.540541,0.919048,0.926316,AdaBoostClassifier,normal,3.633409
3,3,157,83,68,1792,0.654167,0.697778,0.675269,0.928095,0.926316,AdaBoostClassifier,normal,3.633409
4,4,148,125,64,1763,0.542125,0.698113,0.610309,0.910000,0.926316,AdaBoostClassifier,normal,3.633409
5,5,113,121,79,1787,0.482906,0.588542,0.530516,0.904762,0.926316,AdaBoostClassifier,normal,3.633409
6,6,131,50,50,1869,0.723757,0.723757,0.723757,0.952381,0.926316,AdaBoostClassifier,normal,3.633409
7,7,145,58,82,1815,0.714286,0.638767,0.674419,0.933333,0.926316,AdaBoostClassifier,normal,3.633409
8,8,112,46,79,1863,0.708861,0.586387,0.641834,0.940476,0.926316,AdaBoostClassifier,normal,3.633409
9,9,138,115,103,1744,0.545455,0.572614,0.558704,0.896190,0.926316,AdaBoostClassifier,normal,3.633409


In [25]:
temp = a[a['Model']==model]
scaler = temp['Scaler']



'normal'

In [29]:
final_df = pd.DataFrame()

for model in a['Model'].unique():
    temp = a[a['Model']==model].copy()
    scaler = temp['Scaler'].iloc[0]
    temp = temp.drop(['Model','Scaler'],axis=1)
    temp.loc['Total',:] = temp.mean()
    temp['Class'] = temp['Class'].astype(str)
    temp.loc['Total','Class'] = 'Macro'
    temp['Model'] = model
    temp['Scaler'] = scaler
    final_df = pd.concat([final_df,temp])

    


,Class,TP,FP,FN,TN,Precision,Recall,F1,Accuracy,AUC,Time,Model,Scaler
Total,Macro,137.7,72.3,72.3,1817.7,0.668566,0.653456,0.657360,0.931143,0.926316,3.633409,AdaBoostClassifier,normal
Total,Macro,185.1,24.9,24.9,1865.1,0.880480,0.880222,0.879759,0.976286,0.983628,5.545704,BaggingClassifier,normal
Total,Macro,172.4,37.6,37.6,1852.4,0.822206,0.818228,0.818737,0.964190,0.979861,0.058774,BernoulliNB,normal
Total,Macro,183.6,26.4,26.4,1863.6,0.873130,0.871803,0.872083,0.974857,0.983250,4.310342,CalibratedClassifierCV,normal
Total,Macro,148.3,61.7,61.7,1828.3,0.737425,0.700980,0.688915,0.941238,0.949320,0.026720,ComplementNB,normal
Total,Macro,168.2,41.8,41.8,1848.2,0.798688,0.798041,0.798105,0.960190,0.887969,0.928980,DecisionTreeClassifier,normal
Total,Macro,23.2,186.8,186.8,1703.2,0.011048,0.100000,0.019897,0.822095,0.500000,0.004308,DummyClassifier,normal
Total,Macro,149.5,60.5,60.5,1829.5,0.705722,0.706849,0.705822,0.942381,0.837425,0.025834,ExtraTreeClassifier,normal
Total,Macro,197.9,12.1,12.1,1877.9,0.942513,0.941651,0.941923,0.988476,0.997232,1.470992,ExtraTreesClassifier,normal
Total,Macro,117.0,93.0,93.0,1797.0,0.637055,0.549050,0.503498,0.911429,0.912218,0.105149,GaussianNB,normal


In [3]:
b

,Class,TP,FP,FN,TN,Precision,Recall,F1,Accuracy,AUC
0,0,121,12,90,1877,0.909774,0.573460,0.703488,0.951429,0.922674
1,1,137,17,108,1838,0.889610,0.559184,0.686717,0.940476,0.922674
2,2,141,106,60,1793,0.570850,0.701493,0.629464,0.920952,0.922674
3,3,113,179,105,1703,0.386986,0.518349,0.443137,0.864762,0.922674
4,4,119,68,128,1785,0.636364,0.481781,0.548387,0.906667,0.922674
5,5,78,125,84,1813,0.384236,0.481481,0.427397,0.900476,0.922674
6,6,126,38,74,1862,0.768293,0.630000,0.692308,0.946667,0.922674
7,7,160,65,51,1824,0.711111,0.758294,0.733945,0.944762,0.922674
8,8,146,123,66,1765,0.542751,0.688679,0.607069,0.910000,0.922674
9,9,95,131,98,1776,0.420354,0.492228,0.453461,0.890952,0.922674


In [ ]:
import time
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error
import mlflow
import mlflow.sklearn


import mlflow
from sklearn.model_selection import train_test_split
from MLPipeline import apply_scaling,SKLearnModelList

def MLPipeline(df, 
               project_name,
               scaler,
               ml_model_type='regressor',
               target_column='Target',
               test_size=0.2):
    """
    Runs multiple scikit-learn estimators with MLflow tracking.

    Args:
        df (DataFrame): Input dataset.
        project_name (str): MLflow experiment name.
        scaler (str): One of None, 'normal', or 'standard'.
        ml_model_type (str): 'classifier', 'regressor', 'cluster', 'transformer'
        target_column (str): Name of the target column.
        test_size (float): Proportion of data used for testing.

    Returns:
        pd.DataFrame: Summary of model performance.
    """

    # Set MLflow experiment
    #mlflow.set_experiment(project_name)

    # Prepare data
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)

    # Apply scaler
    X_train, X_test, _ = apply_scaling(X_train, X_test, scaler=scaler)

    # Get model list (you must have this function already working)
    sklearn_models_df = SKLearnModelList()

    if len(X_train)<5000:
        model_list = sklearn_models_df[(sklearn_models_df['Dataset Size'].str.contains('small',case=False))&(sklearn_models_df['Estimator Type'].str.contains(ml_model_type,case=False))]
    elif len(X_train)<100000:
        model_list = sklearn_models_df[(sklearn_models_df['Dataset Size'].str.contains('medium',case=False))&(sklearn_models_df['Estimator Type'].str.contains(ml_model_type,case=False))]
    elif len(X_train)>100000:
        model_list = sklearn_models_df[(sklearn_models_df['Dataset Size'].str.contains('large',case=False))&(sklearn_models_df['Estimator Type'].str.contains(ml_model_type,case=False))]

    results = []

    for _, row in model_list.iterrows():
        name = row['Model Name']
        estimator_class = row['Estimator Class']
        print(f'Generating Predicition for {name}, started processing {datetime.datetime.now()}')
        try:
            start_time = time.time()
            model = estimator_class()

            #with mlflow.start_run(run_name=name):
            model.fit(X_train, y_train)
            y_pred = model.predict(X_test)

            if ml_model_type == "classifier":
                metric = accuracy_score(y_test, y_pred)
                mlflow.log_metric("Accuracy", metric)
            else:
                metric = mean_squared_error(y_test, y_pred) ** 0.5
                mlflow.log_metric("RMSE", metric)

                #mlflow.sklearn.log_model(model, name)
                #mlflow.log_param("Model", name)
                #mlflow.log_param("Training Time", round(time.time() - start_time, 2))

            results.append({
                "Model": name,
                "Metric": metric,
                "Time (s)": round(time.time() - start_time, 2)
                })

        except Exception as e:
            print(f"{name} failed: {str(e)}")

    return pd.DataFrame(results)


results_df = MLPipeline(df,
                        project_name='MNIST_ML_Comparison',
                        scaler='normal',
                        ml_model_type='classifier',
                        target_column='Target',
                        test_size=0.2)

results_df

Generating Predicition for AdaBoostClassifier, started processing 2025-05-07 00:05:14.018523
AdaBoostClassifier failed: Could not find experiment with ID 0
Generating Predicition for BaggingClassifier, started processing 2025-05-07 00:05:37.619173
BaggingClassifier failed: Could not find experiment with ID 0
Generating Predicition for BernoulliNB, started processing 2025-05-07 00:06:40.091800
BernoulliNB failed: Could not find experiment with ID 0
Generating Predicition for CalibratedClassifierCV, started processing 2025-05-07 00:06:40.383110
CalibratedClassifierCV failed: Could not find experiment with ID 0
Generating Predicition for CategoricalNB, started processing 2025-05-07 00:08:02.203059
CategoricalNB failed: index 64 is out of bounds for axis 1 with size 1
Generating Predicition for ClassifierChain, started processing 2025-05-07 00:08:03.445802
ClassifierChain failed: ClassifierChain.__init__() missing 1 required positional argument: 'base_estimator'
Generating Predicition for 

/Users/derekdewald/anaconda3/envs/D2/lib/python3.11/site-packages/sklearn/semi_supervised/_label_propagation.py:231: RuntimeWarning: invalid value encountered in divide
  probabilities /= normalizer


LabelPropagation failed: Could not find experiment with ID 0
Generating Predicition for LabelSpreading, started processing 2025-05-07 00:45:23.185455


In [8]:
sklearn_models_df = results_df.copy()

X_train = range(0,400000)

ml_model_type= 'classifier'

if len(X_train)<5000:
    model_list = sklearn_models_df[(sklearn_models_df['Dataset Size'].str.contains('small',case=False))&(sklearn_models_df['Estimator Type'].str.contains(ml_model_type=ml_model_type,case=False))]['Model Name'].tolist()
elif len(X_train)<100000:
    model_list = sklearn_models_df[(sklearn_models_df['Dataset Size'].str.contains('medium|Small',case=False))&(sklearn_models_df['Estimator Type'].str.contains(ml_model_type=ml_model_type,case=False))]['Model Name'].tolist()
elif len(X_train)>100000:
    model_list = sklearn_models_df[(sklearn_models_df['Dataset Size'].str.contains('large|medium|small',case=False))&(sklearn_models_df['Estimator Type'].str.contains(ml_model_type=ml_model_type,case=False))]['Model Name'].tolist()


TypeError: StringMethods.contains() got an unexpected keyword argument 'ml_model_type'

In [7]:

def MLPipeline(df, 
               project_name,
               scaler,
               ml_model_type='regressor',
               target_column='Target',
               test_size=0.2):
    """
    Runs multiple ML algorithms, tracks results with MLflow, and saves models.

    Args:
        df (dataframe)
        project_name (str):
        scaler (str): None, normal,standard
        ml_model_type (str): Option to pronpt all_estimators as to what model type requested.
        classifier, regressor, cluster, transformer

    Returns:
        None (Results are logged in MLflow)
    """
    # Set up MLflow experiment
    mlflow.set_experiment(project_name)

    # Prepare data
    X = df.drop(columns=[target_column])
    y = df[target_column]

    # Train-test split
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=42)
    
    # Apply Scaler as Necessary
    X_train, X_test, scaler = apply_scaling(X_train, X_test, scaler=scaler)
    
    # Get all available models
    model_list = SKLearnModelList(ml_model_type)

    results = []

    for idx, row in model_list.iterrows():
        try:
            name = row['Model Name']
            model = row['Estimator Class']
            start_time = time.time()

             with mlflow.start_run(run_name=m_name):  # Start MLflow run
                 model = model()
                 model.fit(X_train, y_train)  # Train model
                 y_pred = model.predict(X_test)  # Predict

                # Evaluate performance
                if ml_model_type == "classifier":
                    metric = accuracy_score(y_test, y_pred)
                    mlflow.log_metric("Accuracy", metric)
                else:
                    metric = mean_squared_error(y_test, y_pred) ** 0.5  # RMSE manually computed
                    mlflow.log_metric("RMSE", metric)

                # Log model
                mlflow.sklearn.log_model(model, name)

                # Log metadata
                mlflow.log_param("Model", name)
                mlflow.log_param("Training Time", round(time.time() - start_time, 2))

                # Append results
                results.append({
                    "Model": name,
                    "Metric": metric,
                    "Time (s)": round(time.time() - start_time, 2)
                })

        except Exception as e:
            print(f"{name} failed: {str(e)}")  # Handle errors but continue

    # Convert results to DataFrame
    results_df = pd.DataFrame(results)
    results_df = results_df#.sort_values(by="Metric", ascending=(task_type == "regression"))

    return results_df
    


,Model Name,Estimator Class,Full Class Path,Estimator Type,Part_1,Part_2,Part_3,Part_4,Part_5
1,AdaBoostClassifier,<class 'sklearn.ensemble._weight_boosting.AdaB...,sklearn.ensemble._weight_boosting.AdaBoostClas...,classifier,sklearn,ensemble,_weight_boosting,AdaBoostClassifier,None
6,BaggingClassifier,<class 'sklearn.ensemble._bagging.BaggingClass...,sklearn.ensemble._bagging.BaggingClassifier,classifier,sklearn,ensemble,_bagging,BaggingClassifier,None
10,BernoulliNB,<class 'sklearn.naive_bayes.BernoulliNB'>,sklearn.naive_bayes.BernoulliNB,classifier,sklearn,naive_bayes,BernoulliNB,None,None
16,CalibratedClassifierCV,<class 'sklearn.calibration.CalibratedClassifi...,sklearn.calibration.CalibratedClassifierCV,classifier,sklearn,calibration,CalibratedClassifierCV,None,None
17,CategoricalNB,<class 'sklearn.naive_bayes.CategoricalNB'>,sklearn.naive_bayes.CategoricalNB,classifier,sklearn,naive_bayes,CategoricalNB,None,None
18,ClassifierChain,<class 'sklearn.multioutput.ClassifierChain'>,sklearn.multioutput.ClassifierChain,classifier,sklearn,multioutput,ClassifierChain,None,None
20,ComplementNB,<class 'sklearn.naive_bayes.ComplementNB'>,sklearn.naive_bayes.ComplementNB,classifier,sklearn,naive_bayes,ComplementNB,None,None
23,DecisionTreeClassifier,<class 'sklearn.tree._classes.DecisionTreeClas...,sklearn.tree._classes.DecisionTreeClassifier,classifier,sklearn,tree,_classes,DecisionTreeClassifier,None
27,DummyClassifier,<class 'sklearn.dummy.DummyClassifier'>,sklearn.dummy.DummyClassifier,classifier,sklearn,dummy,DummyClassifier,None,None
33,ExtraTreeClassifier,<class 'sklearn.tree._classes.ExtraTreeClassif...,sklearn.tree._classes.ExtraTreeClassifier,classifier,sklearn,tree,_classes,ExtraTreeClassifier,None


In [10]:
c_models = SKLearnModelList('classifier')


  

AdaBoostClassifier <class 'sklearn.ensemble._weight_boosting.AdaBoostClassifier'>
BaggingClassifier <class 'sklearn.ensemble._bagging.BaggingClassifier'>
BernoulliNB <class 'sklearn.naive_bayes.BernoulliNB'>
CalibratedClassifierCV <class 'sklearn.calibration.CalibratedClassifierCV'>
CategoricalNB <class 'sklearn.naive_bayes.CategoricalNB'>
ClassifierChain <class 'sklearn.multioutput.ClassifierChain'>
ComplementNB <class 'sklearn.naive_bayes.ComplementNB'>
DecisionTreeClassifier <class 'sklearn.tree._classes.DecisionTreeClassifier'>
DummyClassifier <class 'sklearn.dummy.DummyClassifier'>
ExtraTreeClassifier <class 'sklearn.tree._classes.ExtraTreeClassifier'>
ExtraTreesClassifier <class 'sklearn.ensemble._forest.ExtraTreesClassifier'>
FixedThresholdClassifier <class 'sklearn.model_selection._classification_threshold.FixedThresholdClassifier'>
GaussianNB <class 'sklearn.naive_bayes.GaussianNB'>
GaussianProcessClassifier <class 'sklearn.gaussian_process._gpc.GaussianProcessClassifier'>
Gra

In [8]:
import inspect

model_params = inspect.signature('AdaBoostClassifier').parameters

TypeError: 'AdaBoostClassifier' is not a callable object